# Using an AutoML library to obtain results : fbprohret

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
base_test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
df.head()

# Baseline Model

In [ ]:
train_df_baseline = df.rename(columns={"datetime": "ds", "count":"y"})
train_df_baseline.drop(columns = ['registered', 'casual'])
test_df_baseline = base_test_df.rename(columns={"datetime": "ds"})

In [ ]:
m = Prophet(changepoint_prior_scale=1.5)
m.fit(train_df_baseline)

In [ ]:
forecast = m.predict(test_df_baseline)
forecast

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
def create_submission(forecast):
    result = forecast[['ds', 'trend']]
    result = result.rename(columns = { 'ds': 'datetime', 'trend':'count' })
    result.set_index("datetime", inplace = True) 
    result.to_csv('submission.csv')

In [ ]:
create_submission(forecast)

We get bad result : 1.07

# Two Models : one for casual and for registered

In [ ]:
train_df_casual = df.rename(columns={"datetime": "ds", "casual":"y"})
test_df_casual = base_test_df.rename(columns={"datetime": "ds"})

In [ ]:
train_df_casual

In [ ]:
train_df_casual = train_df_casual.drop(columns = ['registered', 'count'])

In [ ]:
train_df_casual

In [ ]:
casual_model = Prophet()
casual_model.fit(train_df_casual)
forecast = casual_model.predict(test_df_casual)
forecast

In [ ]:
casual_model.plot(forecast)

We can see a year temporality, let's do the same thing with registered

In [ ]:
train_df_registered = df.rename(columns={"datetime": "ds", "registered":"y"})
test_df_registered = base_test_df.rename(columns={"datetime": "ds"})
train_df_registered = train_df_registered.drop(columns = ['casual', 'count'])

registered_model = Prophet()
registered_model.fit(train_df_registered)
forecast_registered = registered_model.predict(test_df_registered)
registered_model.plot(forecast_registered)

### Mix both models

In [ ]:
forecast

In [ ]:
forecast_registered

In [ ]:
ultimate_forecast = forecast_registered['trend'] + forecast['trend']

In [ ]:
ultimate_forecast

In [ ]:
frame = {'count': ultimate_forecast} 
result = pd.DataFrame(frame)

In [ ]:
result['datetime'] = base_test_df.datetime
result.set_index("datetime", inplace = True) 
result.to_csv('submission.csv')

The results are even worse ! 1.57

In conclusion, we can see that this AutoML library is **very** easy to use but it has way lower results than basic models like random forest or gradient boosting trained in cross-validation. 
It can be a good starting point for non-data scientist wanting to have a basic model.